In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
database = pd.read_csv('COBRA-YTD2017.csv')

In [3]:
database.rename(columns={'UC2 Literal' : 'Crime', 'occur_time': 'time', 'MaxOfnum_victims': 'victims', 'Avg Day': 'day'}, inplace=True)
database.head()

,MI_PRINX,offense_id,rpt_date,occur_date,time,poss_date,poss_time,beat,apt_office_prefix,apt_office_num,...,dispo_code,victims,Shift,day,loc_type,Crime,neighborhood,npu,x,y
0,8924155,173650072,12/31/2017,12/30/2017,23:15:00,12/31/2017,00:30:00,510,NaN,NaN,...,NaN,2.0,Morn,Sat,13.0,LARCENY-FROM VEHICLE,Downtown,M,-84.38013,33.75582
1,8924156,173650102,12/31/2017,12/18/2017,13:00:00,12/30/2017,22:00:00,501,NaN,NaN,...,NaN,1.0,Unk,Unk,13.0,LARCENY-FROM VEHICLE,Home Park,E,-84.39745,33.78674
2,8924157,173650144,12/31/2017,12/30/2017,22:01:00,12/31/2017,01:00:00,303,NaN,NaN,...,NaN,1.0,Morn,Sat,18.0,LARCENY-FROM VEHICLE,Mechanicsville,V,-84.39486,33.73760
3,8924158,173650149,12/31/2017,12/30/2017,20:00:00,12/31/2017,01:06:00,507,NaN,NaN,...,NaN,1.0,Eve,Sat,18.0,LARCENY-FROM VEHICLE,Castleberry Hill,M,-84.39887,33.75156
4,8924159,173650159,12/31/2017,12/31/2017,00:41:00,12/31/2017,00:48:00,409,NaN,NaN,...,NaN,2.0,Morn,Sun,18.0,LARCENY-FROM VEHICLE,Adams Park,R,-84.46522,33.72146


In [4]:
# NOTE : I removed rows where victims are only 1, it screws up the results.

# database.day.unique()
arr = [23, 24, 21, 19, 27, 17, 15]
database = database[database.victims != 23]
database = database[database.victims != 24]
database = database[database.victims != 21]
database = database[database.victims != 19]
database = database[database.victims != 27]
database = database[database.victims != 17]
database = database[database.victims != 15]
database.Crime.value_counts()

LARCENY-FROM VEHICLE    9837
LARCENY-NON VEHICLE     6586
AUTO THEFT              3197
BURGLARY-RESIDENCE      2635
AGG ASSAULT             2023
ROBBERY-PEDESTRIAN      1126
BURGLARY-NONRES          758
RAPE                     226
ROBBERY-COMMERCIAL       157
ROBBERY-RESIDENCE        132
HOMICIDE                  75
Name: Crime, dtype: int64

In [5]:
# NOTE: Clean up database and remove strings basically.

database['Crime'] = database['Crime'].replace({'LARCENY-FROM VEHICLE': '1',
                          'ROBBERY-PEDESTRIAN': '2',
                          'BURGLARY-RESIDENCE': '3',
                          'AUTO THEFT': '4',
                          'BURGLARY-NONRES': '5',
                          'ROBBERY-RESIDENCE': '6',
                          'LARCENY-NON VEHICLE': '7',
                          'AGG ASSAULT': '8',
                            'RAPE': '9',
                          'ROBBERY-COMMERCIAL': '10',
                          'HOMICIDE': '11'})

database['day'] = database['day'].replace({'Mon': '1',
                                          'Tue': '2',
                                          'Wed': '3',
                                          'Thu': '4',
                                          'Fri': '5',
                                          'Sat': '6',
                                          'Sun': '7',
                                          'Unk': '0'})

In [6]:
# NOTE : Gotta convert everything to int for train_test_split

database['time'] = pd.to_datetime(database.time)
# # TODO: Figure out how to plot a scatter_matrix

# # from pandas.plotting import scatter_matrix

# # database['occur_time'] = pd.to_datetime(database['occur_time'], errors='coerce')

# # database['occur_time'] = database['occur_time'].dt.time

database['hour'] = database.time.dt.hour
# database['hour'].value_counts()

database['victims'] = database['victims'].fillna(0)
database.victims = database.victims.astype(int)
database.hour = database.victims.astype(int)
database.x = database.x.astype(int)
database.y = database.y.astype(int)
database.day = database.day.astype(int)

database.dtypes

MI_PRINX                      int64
offense_id                    int64
rpt_date                     object
occur_date                   object
time                 datetime64[ns]
poss_date                    object
poss_time                    object
beat                          int64
apt_office_prefix            object
apt_office_num               object
location                     object
MinOfucr                      int64
MinOfibr_code                object
dispo_code                   object
victims                       int32
Shift                        object
day                           int32
loc_type                    float64
Crime                        object
neighborhood                 object
npu                          object
x                             int32
y                             int32
hour                          int32
dtype: object

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_all = database.drop(['Crime'], 1)
y_all = database['Crime']

X_all = database[['hour', 'day', 'x', 'y', 'victims']]

X_all = X_all.fillna(0)

X_all = scaler.fit_transform(X_all)

In [131]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state = 2)

In [9]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [ ]:
classif = OneVsRestClassifier(SVC(kernel = 'linear'))
classif.fit(X_train, y_train)

y_pred = classif.predict(X_train)

print(sum(y_train == y_pred) / float(len(y_pred)))

y_pred = classif.predict(X_test)

print(sum(y_test == y_pred) / float(len(y_pred)))